<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP_2022/blob/main/02-Machine-Learning/notebooks/13-Clustering-Grafos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clusterizando nodos de un grafo

En esta notebook se muestra un ejemplo de clusterización de nodos de grafos usando una matriz de features para los nodos.

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

## 1. El conjunto de datos 

Este dataset del módulo networkx es contiene información de los miembros de un club de karate universitario. El grafo representa la presencia o ausencia de lazos entre los miembros del club. Los pesos de las aristas indican la fuerza del lazo entre miembros (el número de situaciones dentro y fuera del club en las cuales las interacciones occurren).

Estos datos primero se usaron para explicar la ruptura de este grupo como consecuencia de las disuputas entre miembros. 

[Link](http://vlado.fmf.uni-lj.si/pub/networks/data/Ucinet/UciData.htm#zachary) del dataset.

In [ ]:
kn = nx.karate_club_graph()

num_nodes = kn.number_of_nodes()
print(f"Number of nodes: {str(num_nodes)}")
num_edges = kn.number_of_edges()
print(f"Number of edges: {str(num_edges)}")

Graficamos el grafo:

In [ ]:
plt.figure(figsize=(12,10))
nx.draw_networkx(kn, edge_color='silver', with_labels=True, font_color='white')
plt.axis('off')
plt.show()

Calculamos la centralidad de los nodos en un diccionario:

In [4]:
deg_centrality_dict = nx.degree_centrality(kn).copy()

Ordenamos el diccionario por los valores de centralidad

In [5]:
deg_centrality_dict_sorted = dict(sorted(deg_centrality_dict.items(), 
                                         key=lambda x:x[1],reverse=True))
print(deg_centrality_dict_sorted)

{33: 0.5151515151515151, 0: 0.48484848484848486, 32: 0.36363636363636365, 2: 0.30303030303030304, 1: 0.2727272727272727, 3: 0.18181818181818182, 31: 0.18181818181818182, 8: 0.15151515151515152, 13: 0.15151515151515152, 23: 0.15151515151515152, 5: 0.12121212121212122, 6: 0.12121212121212122, 7: 0.12121212121212122, 27: 0.12121212121212122, 29: 0.12121212121212122, 30: 0.12121212121212122, 4: 0.09090909090909091, 10: 0.09090909090909091, 19: 0.09090909090909091, 24: 0.09090909090909091, 25: 0.09090909090909091, 28: 0.09090909090909091, 9: 0.06060606060606061, 12: 0.06060606060606061, 14: 0.06060606060606061, 15: 0.06060606060606061, 16: 0.06060606060606061, 17: 0.06060606060606061, 18: 0.06060606060606061, 20: 0.06060606060606061, 21: 0.06060606060606061, 22: 0.06060606060606061, 26: 0.06060606060606061, 11: 0.030303030303030304}


Dibujamos los 5 nodos con centralidad más alta

In [ ]:
top_num = 5
top_nodes = list(deg_centrality_dict_sorted.keys())[:top_num]
print(f"Los {top_num} nodos con centralidad más alta: {top_nodes}")

colores = ['red' if x in top_nodes else 'blue' for x in kn.nodes() ]

fig = plt.figure(figsize=(12,10))
nx.draw_spring(kn,node_color=colores,with_labels=True)
fig.show()

## 2. Extrayendo features

La matriz de adyacencia de un grafo finito es una matriz cuadrada utilizada para representar el grafo. Los elementos $(i,j)$ de la matriz indican si los pares de nodos $i$ y $j$ son adyacentes o no en el grafo (con valores 0 y 1). En caso de ser un grafo pesado, el valor de la entrada $(i,j)$ es el peso de la arista.  

El grado de cada nodo, en el caso de un grafo no pesado, es la cantidad de aristas que salen (o entran) del nodo. En el caso de un grafo pesado, es la suma de los pesos de las aristas que salen (o entran).

La matriz laplaciana es otra matriz que representa el grafo. Se define como

$$L = D - A,$$

donde $D$ es la matriz que contiene el grado de cada nodo en la diagonal y $A$, la matriz de adyacencia.

Tanto la [matriz de adyancencia](https://networkx.org/documentation/stable/reference/generated/networkx.linalg.graphmatrix.adjacency_matrix.html), como la [matriz Laplaciana](https://networkx.org/documentation/stable/reference/generated/networkx.linalg.laplacianmatrix.laplacian_matrix.html), se obtienen con métodos de networkx.

In [ ]:
A = nx.adjacency_matrix(kn)
print("Matriz de adyacencia:")
print(A.todense())
print('-'*50)
print("Grado (con peso) de cada nodo:")
print(kn.degree(weight='weight'))
L = nx.laplacian_matrix(kn).astype(float)
print('-'*50)
print("Matriz Laplaciana:")
print(L.todense())


Obtenemos los $k$ primeros eigenvalores y eigenvectores.

In [ ]:
import scipy as sp

eigenvalues, eigenvectors = sp.sparse.linalg.eigsh(L, k = 3, which='SM')
print(eigenvalues.shape,eigenvectors.shape)

Obtenemos una representación espectral de cada nodo con 3 componentes. Esta es la matriz de features que usaremos, en esta, cada nodo está representado por 3 features.

In [ ]:
X = eigenvectors*eigenvalues
print(X.shape)
print(X)

## 3. Clustering

Usaremos la matriz $X$ de features para hacer clustering sobre los nodos

In [10]:
# def get_cmap(n, name='plasma'):
#     '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
#     RGB color; the keyword argument name must be a standard mpl colormap name.'''
#     return plt.cm.get_cmap(name, n)

# print([get_cmap(k) for k in range(3)])

Esta es una función auxiliar para obtener una lista de $n$ colores muy diferentes, los cuales usaremos al graficar los clusters

In [11]:
import random
import numpy as np

def get_k_colors(k=3):
    return [(np.cos(np.pi*j/(k*2)),random.uniform(0, 1),j/k) for j in range(k)]

In [ ]:
from sklearn.cluster import KMeans

k = 3
clusters_colors = get_k_colors(k)

model = KMeans(init='k-means++', n_clusters=k, n_init=10)
model.fit_predict(X)
clusters = model.labels_

fig = plt.subplots(1, figsize=(12,10))
node_colors = [clusters_colors[clusters[v]] for v in kn.nodes()]
nx.draw(kn, node_color=node_colors, with_labels='True')

Podemos usar el elbow value (ya que estamos con K-means) o el valor de silueta para elegir un buen valor para el número de clusters

In [ ]:
from sklearn.metrics import silhouette_score

sils = []
intertias = []

ks = list(range(2,21))

for k in ks:
    model = KMeans(init='k-means++', n_clusters=k, n_init=10,random_state=26)
    model.fit_predict(X)
    clusters = model.labels_
    intertias.append(model.inertia_)
    sils.append(silhouette_score(X,clusters))


fig, axs = plt.subplots(1,2,figsize=(10,6))
axs[0].title.set_text('Inercias')
axs[0].plot(ks,intertias)
axs[0].scatter(ks,intertias)
axs[0].set_xticks(ks)
axs[1].title.set_text('Valor de siluetas')
axs[1].plot(ks,sils)
axs[1].scatter(ks,sils)
axs[1].set_xticks(ks)
fig.show()

In [ ]:
from sklearn.cluster import KMeans

k = 5
clusters_colors = get_k_colors(k)

model = KMeans(init='k-means++', n_clusters=k, n_init=10)
model.fit_predict(X)
clusters = model.labels_

fig = plt.subplots(1, figsize=(12,10))
node_colors = [clusters_colors[clusters[v]] for v in kn.nodes()]
nx.draw(kn, node_color=node_colors, with_labels='True')

Ahora usemos `AgglomerativeClustering`

In [ ]:
from sklearn.cluster import AgglomerativeClustering

k = 5
clusters_colors = get_k_colors(k)

model = AgglomerativeClustering(n_clusters=k) #, affinity='precomputed', linkage='complete')
model.fit(X)
clusters = model.labels_

fig = plt.subplots(1, figsize=(12,10))
node_colors = [clusters_colors[clusters[v]] for v in kn.nodes()]
nx.draw(kn, node_color=node_colors, with_labels='True')

Usemos SpectralClustering y DBSCAN

In [ ]:
from sklearn.cluster import SpectralClustering

k = 5
clusters_colors = get_k_colors(k)

model = SpectralClustering(n_clusters=k)
model.fit(X)
clusters = model.labels_

print(f"{len(np.unique(clusters))} clusters encontrados")

plt.figure(figsize=(12,10))
node_colors = [clusters_colors[clusters[v]] for v in kn.nodes()]
nx.draw(kn, node_color=node_colors, with_labels='True')

In [ ]:
from sklearn.cluster import DBSCAN

model = DBSCAN(eps=0.1)
model.fit(X)
clusters = model.labels_

num_clusters = len(np.unique(clusters))
print(f"{num_clusters} clusters encontrados")
clusters_colors = get_k_colors(num_clusters)

plt.figure(figsize=(10,10))
node_colors = [clusters_colors[clusters[v]] for v in kn.nodes()]
nx.draw(kn, node_color=node_colors, with_labels='True')

### Usando métricas predefinidas

La matriz de adyacencia no contiene la información necesaria para distinguir clusters.

In [ ]:
print(f"Matriz de adyancencia:\n{A.todense()}")

In [ ]:
from sklearn.cluster import AgglomerativeClustering

k = 3
clusters_colors = get_k_colors(k)

model = AgglomerativeClustering(n_clusters=k, affinity='precomputed', linkage='complete')
model.fit(A.toarray())
clusters = model.labels_

print(f"{len(np.unique(clusters))} clusters encontrados")

plt.figure(figsize=(12,10))
node_colors = [clusters_colors[clusters[v]] for v in kn.nodes()]
nx.draw(kn, node_color=node_colors, with_labels='True')

⭕ ¿Por qué no produce buenos resultados?

Por otro lado, la matriz de distancias entre nodos, puede dar cuenta de algunos clusters. La usaremos como matriz de distancias para usar la opción `precomputed`.

In [ ]:
longs = dict(nx.all_pairs_shortest_path_length(kn))
n_nodes = kn.number_of_nodes()

distancias = np.zeros(shape=(n_nodes,n_nodes))

for i in range(n_nodes):
    for j in range(n_nodes):
        distancias[i,j] = longs[i][j]
print(f"Matriz de distancias:\n\n{distancias}")

In [ ]:
from sklearn.cluster import AgglomerativeClustering

k = 5
clusters_colors = get_k_colors(k)

model = AgglomerativeClustering(n_clusters=k, affinity='precomputed', linkage='complete')
model.fit(distancias)
clusters = model.labels_

print(f"{len(np.unique(clusters))} clusters encontrados")

plt.figure(figsize=(12,10))
node_colors = [clusters_colors[clusters[v]] for v in kn.nodes()]
nx.draw(kn, node_color=node_colors, with_labels='True')